In [1]:
import math
import numpy as np
import scipy.spatial
import scipy.optimize

In [2]:
corners = np.array([
    [-.5, -.5, -.5],
    [+.5, -.5, -.5],
    [+.5, +.5, -.5],
    [-.5, +.5, -.5],
    [-.5, -.5, +.5],
    [+.5, -.5, +.5],
    [+.5, +.5, +.5],
    [-.5, +.5, +.5],
])

faces = np.array([
    [.5, 0, 0],
    [0, .5, 0],
    [0, 0, .5]
])

In [15]:
def roll_matrix(angle):
    R =  np.array([
        [1, 0, 0],
        [0, np.cos(angle), -np.sin(angle)],
        [0, np.sin(angle), np.cos(angle)]
    ])
    return R

def pitch_matrix(angle):
    R =  np.array([
        [np.cos(angle), 0, np.sin(angle)],
        [0, 1, 0],
        [-np.sin(angle), 0, np.cos(angle)]
    ])
    return R

def yaw_matrix(angle):
    R = np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]
    ])
    return R
    
def rotation_matrix(roll, pitch, yaw):
    R = np.dot(np.dot(yaw_matrix(yaw), pitch_matrix(pitch)), roll_matrix(roll))
    return R

def rotate_points(points, roll, pitch, yaw):
    R = rotation_matrix(roll, pitch, yaw)
    rotated_points = np.dot(points, R)
    return rotated_points

def proj_area(corners):
    #  plane = np.array([[1, 0, 0], [0, 0, 1]])
    #  projected_corners = np.dot(corners, plane.T)
    projected_corners = corners[:,:2]  # Simplify, since the plane is the y-plane
    # Find area of convex hull
    hull = scipy.spatial.ConvexHull(projected_corners)
    return hull.volume

def func_min(angles, corners, specified_area):
    rotated_corners = rotate_points(corners, angles[0], angles[1], 0)
    area = proj_area(rotated_corners)
    return abs(area - specified_area)

In [16]:
R = rotation_matrix(0,0,0)
R

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [17]:
rotate_points(corners, 0, 0, 0)

array([[-0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5],
       [ 0.5,  0.5, -0.5],
       [-0.5,  0.5, -0.5],
       [-0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5],
       [ 0.5,  0.5,  0.5],
       [-0.5,  0.5,  0.5]])

In [18]:
proj_area(rotate_points(corners, 0, 0, 0))

1.0

In [19]:
func_min([0,0], corners, 1)

0.0

In [20]:
np.sqrt(2)

1.4142135623730951

In [21]:
x0 = [0, 0]
specified_area = np.sqrt(2)
result = scipy.optimize.minimize(func_min, x0, args=(corners, specified_area),
                                 method='Nelder-Mead', options={'ftol': 1e-14})
roll, pitch = result.x
roll, pitch

(0.16144353290167968, 0.32983859878367905)

In [22]:
result

 final_simplex: (array([[0.16144353, 0.3298386 ],
       [0.16144348, 0.32983866],
       [0.1614435 , 0.32983865]]), array([1.11022302e-15, 3.99680289e-15, 8.88178420e-15]))
           fun: 1.1102230246251565e-15
       message: 'Optimization terminated successfully.'
          nfev: 171
           nit: 95
        status: 0
       success: True
             x: array([0.16144353, 0.3298386 ])

In [24]:
proj_area(rotate_points(corners, roll, pitch, 0)) - specified_area

1.1102230246251565e-15

In [28]:
rotated_faces = rotate_points(faces, roll, pitch, 0)

In [29]:
for face in rotated_faces:
    print("{} {} {}".format(face[0], face[1], face[2]))

0.47304731607235745 0.026031573695093697 0.15983927529770675
0.0 0.49349813689839306 -0.08037156759585402
-0.16194516589493857 0.0760391086794936 0.4668959382929873


In [30]:
rotated_faces

array([[ 0.47304732,  0.02603157,  0.15983928],
       [ 0.        ,  0.49349814, -0.08037157],
       [-0.16194517,  0.07603911,  0.46689594]])